# Fine tuning T5 summarization

Base on code from https://github.com/huggingface/notebooks/blob/master/examples/summarization.ipynb

In [1]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/furyhawk/text_summarization/raw/master'
    os.system(f'wget {GIT_ROOT}/notebooks/setup.py')

%run -i setup.py


You are working on a local system.
Files will be searched relative to "..".


In [2]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# path to import blueprints packages
sys.path.append(BASE_DIR + '/packages')

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [3]:

! pip install datasets transformers rouge-score nltk

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [4]:
from huggingface_hub import notebook_login

#notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [5]:
#!apt install git-lfs

In [6]:
#!git clone https://github.com/jessevig/bertviz.git

In [7]:

import os.path
import pandas as pd
import numpy as np

## Load BBC News summary dataset

In [8]:
root_path = f'../data/BBC News Summary'


# root_path = f'/kaggle/input/bbc-news-summary/BBC News Summary'


def loadDataset(root_path):

    types_of_articles = ['business',
                         'entertainment', 'politics', 'sport', 'tech']
    df = pd.DataFrame(columns=['title', 'article', 'summary'])

    for type_of_article in types_of_articles:
        # type_of_article = 'business'  # entertainment, politices, sport, tech
        num_of_article = len(os.listdir(
            f"{root_path}/News Articles/{type_of_article}"))

        print(f'"Reading {type_of_article} articles"')
        dataframe = pd.DataFrame(columns=['title', 'article', 'summary'])

        for i in tqdm(range(num_of_article)):
            with open(f'{root_path}/News Articles/{type_of_article}/{(i+1):03d}.txt', 'r', encoding="utf8", errors='ignore') as f:
                article = f.read().partition("\n")
            with open(f'{root_path}/Summaries/{type_of_article}/{(i+1):03d}.txt', 'r', encoding="utf8", errors='ignore') as f:
                summary = f.read()

            dataframe.loc[i] = [article[0], article[2].replace(
                '\n', ' ').replace('\r', ''), summary]

        df = df.append(dataframe, ignore_index=True)

    return df


In [9]:
df = loadDataset(root_path)

"Reading business articles"


  0%|          | 0/510 [00:00<?, ?it/s]

"Reading entertainment articles"


  0%|          | 0/386 [00:00<?, ?it/s]

"Reading politics articles"


  0%|          | 0/417 [00:00<?, ?it/s]

"Reading sport articles"


  0%|          | 0/511 [00:00<?, ?it/s]

"Reading tech articles"


  0%|          | 0/401 [00:00<?, ?it/s]

In [10]:
df.head()

,title,article,summary
0,Ad sales boost Time Warner profit,"Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Goo...","TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09b..."
1,Dollar gains on Greenspan speech,The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise. And Alan Greenspan highlighted the US g...,The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.China's currency remains pegged to the dol...
2,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (£479m) loan. State-owned Rosneft bought the Yugansk unit for $9.3bn in a s...,Yukos' owner Menatep Group says it will ask Rosneft to repay a loan that Yugansk had secured on its assets.State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part...
3,High fuel prices hit BA's profits,"British Airways has blamed high fuel prices for a 40% drop in profits. Reporting its results for the three months to 31 December 2004, the airline made a pre-tax profit of £75m ($141m) compared ...","Rod Eddington, BA's chief executive, said the results were ""respectable"" in a third quarter when fuel costs rose by £106m or 47.3%.To help offset the increased price of aviation fuel, BA last year..."
4,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Domecq have risen on speculation that it could be the target of a takeover by France's Pernod Ricard. Reports in the Wall Street Journal and the Financia...,"Pernod has reduced the debt it took on to fund the Seagram purchase to just 1.8bn euros, while Allied has improved the performance of its fast-food chains.Shares in UK drinks and food firm Allied ..."


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    2225 non-null   object
 1   article  2225 non-null   object
 2   summary  2225 non-null   object
dtypes: object(3)
memory usage: 52.3+ KB


In [12]:
#df.to_csv('bbc.csv')

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [13]:
import transformers

print(transformers.__version__)

4.11.3


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

# Fine-tuning a model on a summarization task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a summarization task. We will use the [BBC dataset](https://www.kaggle.com/pariza/bbc-news-summary) which contains BBC News Summary.

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [14]:
model_checkpoint = "t5-base"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint. 

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [15]:
from datasets import Dataset, DatasetDict, load_metric
raw_datasets = Dataset.from_pandas(df)  # Load from dataframe created earlier

metric = load_metric("rouge")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [16]:
raw_datasets

Dataset({
    features: ['title', 'article', 'summary'],
    num_rows: 2225
})

In [17]:
raw_datasets.info 

DatasetInfo(description='', citation='', homepage='', license='', features={'title': Value(dtype='string', id=None), 'article': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

Whether to use news headline or summary as label

In [18]:
# raw_datasets = raw_datasets.remove_columns("title")

## Splitting the dataset in train and test split

In [19]:
# 90% train, 10% test + validation
train_testvalid = raw_datasets.train_test_split(test_size=0.1)
# Split the 10% test + validation in half test, half validation
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
raw_datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"]})

raw_datasets


DatasetDict({
    train: Dataset({
        features: ['title', 'article', 'summary'],
        num_rows: 2002
    })
    test: Dataset({
        features: ['title', 'article', 'summary'],
        num_rows: 112
    })
    valid: Dataset({
        features: ['title', 'article', 'summary'],
        num_rows: 111
    })
})

To access an actual element, you need to select a split first, then give an index:

In [20]:
raw_datasets["train"][0]

{'title': 'Monsanto fined $1.5m for bribery',
 'article': ' The US agrochemical giant Monsanto has agreed to pay a $1.5m (£799,000) fine for bribing an Indonesian official.  Monsanto admitted one of its employees paid the senior official two years ago in a bid to avoid environmental impact studies being conducted on its cotton. In addition to the penalty, Monsanto also agreed to three years\' close monitoring of its business practices by the American authorities. It said it accepted full responsibility for what it called improper activities.  A former senior manager at Monsanto directed an Indonesian consulting firm to give a $50,000 bribe to a high-level official in Indonesia\'s environment ministry in 2002. The manager told the company to disguise an invoice for the bribe as "consulting fees".  Monsanto was facing stiff opposition from activists and farmers who were campaigning against its plans to introduce genetically-modified cotton in Indonesia. Despite the bribe, the official di

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [21]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [22]:
show_random_elements(raw_datasets["train"])

,title,article,summary
0,Web photo storage market hots up,"An increasing number of firms are offering web storage for people with digital photo collections. Digital cameras were the hot gadget of Christmas 2004 and worldwide sales of the cameras totalled $24bn last year. Many people's hard drives are bulging with photos and services which allow them to store and share their pictures online are becoming popular. Search firms such as Google are also offering more complex tools for managing personal photo libraries. Photo giants such as Kodak offer website storage which manages photo collections, lets users edit pictures online and provides print-ordering services. Some services, such as Kodak's Ofoto and Snapfish, offer unlimited storage space but they do require users to buy some prints online. Other sites, such as Pixagogo, charge a monthly fee. Marcus Hawkins, editor of Digital Camera magazine, said: ""As file sizes of pictures increase, storage becomes a problem. ""People are using their hard drives, backing up on CD and DVD and now they are using online storage solutions. ""They are a place to store pictures, to share their pictures with families and friends and they can print out their photos."" While many of the services are aimed at the amateur and casual digital photographer, other websites are geared up for enthusiasts who want to share tips and information. Photosig is an online community of photographers who can critique each other's work. On Tuesday, Google released free software for organising and finding digital photos stored on a computer's hard drive. The tool, called Picasa, automatically detects photos as they are added to a PC - whether sent via e-mail or transferred from a digital camera. The software includes tools for restoring colour and removing red eye, as well as sharpening images. Photos can then be uploaded to sites such as Ofoto. Many people use the sites to edit and improve their favourite photographs before ordering prints. Mr Hawkins added: ""The growth area is that you can order your prints online. Friends and family can also access pictures you want them to see and they can print them out too. ""Rather than just a place to dump your pictures, it's about sharing them."" The vast majority of pictures remain on a PC's hard drive, which is why search tools, such as those offered by Google, become increasingly important. But some historians and archivists are concerned that the need for perfect pictures will mean that those poor quality prints which offered a tantilising glimpse of the past may disappear forever. ""It's one thing taking pictures, it's another finding them,"" said Mr Hawkins. ""But this is the same problem that has always existed - how many of us have photos in wallets tucked away somewhere?""","Many people's hard drives are bulging with photos and services which allow them to store and share their pictures online are becoming popular.Photo giants such as Kodak offer website storage which manages photo collections, lets users edit pictures online and provides print-ordering services.""They are a place to store pictures, to share their pictures with families and friends and they can print out their photos.""An increasing number of firms are offering web storage for people with digital photo collections.Marcus Hawkins, editor of Digital Camera magazine, said: ""As file sizes of pictures increase, storage becomes a problem.On Tuesday, Google released free software for organising and finding digital photos stored on a computer's hard drive.Some services, such as Kodak's Ofoto and Snapfish, offer unlimited storage space but they do require users to buy some prints online.Mr Hawkins added: ""The growth area is that you can order your prints online.The tool, called Picasa, automatically detects photos as they are added to a PC - whether sent via e-mail or transferred from a digital camera.Friends and family can also access pictures you want them to see and they can print them out too."
1,To

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [23]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [24]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [25]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) # t5-base

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

Load some required javascript libraries for displaying visualization in notebook

You can directly call this tokenizer on one sentence or a pair of sentences:

In [26]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [27]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [28]:
sentence = "He didnt want to talk about cells on the cell phone because he considered it boring"
inputs = tokenizer.encode(sentence, return_tensors='pt', add_special_tokens=True) # return PyTorch tensors
tokens = tokenizer.convert_ids_to_tokens(list(inputs[0])) # Extract sample of batch index 0 from inputs list of lists
print(tokens)

['▁He', '▁didn', 't', '▁want', '▁to', '▁talk', '▁about', '▁cells', '▁on', '▁the', '▁cell', '▁phone', '▁because', '▁', 'he', '▁considered', '▁it', '▁boring', '</s>']


If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [29]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [30]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    # x var for summarization. Column article.
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets y. We are using column summary as label.
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [31]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[21603, 10, 37, 837, 3, 9, 3844, 14676, 6079, 2963, 7, 288, 32, 65, 4686, 12, 726, 3, 9, 1514, 16593, 51, 41, 19853, 4440, 23938, 61, 1399, 21, 3, 2160, 115, 53, 46, 9995, 29, 2314, 5, 2963, 7, 288, 32, 10246, 80, 13, 165, 1652, 1866, 8, 2991, 2314, 192, 203, 977, 16, 3, 9, 6894, 12, 1792, 3262, 1113, 2116, 271, 4468, 30, 165, 7282, 5, 86, 811, 12, 8, 10736, 6, 2963, 7, 288, 32, 92, 4686, 12, 386, 203, 31, 885, 4891, 13, 165, 268, 2869, 57, 8, 797, 5779, 5, 94, 243, 34, 4307, 423, 3263, 21, 125, 34, 718, 22187, 1087, 5, 71, 1798, 2991, 2743, 44, 2963, 7, 288, 32, 6640, 46, 9995, 29, 9157, 1669, 12, 428, 3, 9, 29788, 3, 2160, 346, 12, 3, 9, 306, 18, 4563, 2314, 16, 9995, 31, 7, 1164, 8409, 16, 4407, 5, 37, 2743, 1219, 8, 349, 12, 31993, 46, 10921, 21, 8, 3, 2160, 346, 38, 96, 29492, 53, 3051, 1280, 2963, 7, 288, 32, 47, 5008, 14537, 8263, 45, 19053, 11, 7208, 113, 130, 2066, 53, 581, 165, 1390, 12, 4277, 6472, 1427, 18, 7360, 3676, 7282, 16, 9995, 5, 3, 4868, 8, 3, 2160, 

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [32]:
# raw_datasets['train'] = raw_datasets['train'].shard(num_shards=100, index=3)
# raw_datasets['validation'] = raw_datasets['validation'].shard(num_shards=100, index=3)
# raw_datasets['test'] = raw_datasets['test'].shard(num_shards=100, index=3)
print(raw_datasets.num_rows)

{'train': 2002, 'test': 112, 'valid': 111}


In [33]:

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [34]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [35]:
batch_size = 12
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-bbc",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/t5-finetuned-xsum"` or `"huggingface/t5-finetuned-xsum"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [36]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [37]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [38]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [39]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

W:\workspace\text_summarization\notebooks\t5-base-finetuned-bbc is already a clone of https://huggingface.co/furyhawk/t5-base-finetuned-bbc. Make sure you pull the latest changes with `repo.git_pull()`.


We can now finetune our model by just calling the `train` method:

In [40]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\furyx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: summary, title, article.
***** Running training *****
  Num examples = 2002
  Num Epochs = 1
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 167


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.260809,20.619600,15.252500,19.093400,19.271600,19.000000


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: summary, title, article.
***** Running Evaluation *****
  Num examples = 111
  Batch size = 12


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=167, training_loss=0.8271075951125093, metrics={'train_runtime': 25534.2229, 'train_samples_per_second': 0.078, 'train_steps_per_second': 0.007, 'total_flos': 2474367178752000.0, 'train_loss': 0.8271075951125093, 'epoch': 1.0})

You can now upload the result of the training to the Hub, just execute this instruction:

In [42]:
trainer.push_to_hub()

Saving model checkpoint to t5-base-finetuned-bbc
Configuration saved in t5-base-finetuned-bbc\config.json
Model weights saved in t5-base-finetuned-bbc\pytorch_model.bin
tokenizer config file saved in t5-base-finetuned-bbc\tokenizer_config.json
Special tokens file saved in t5-base-finetuned-bbc\special_tokens_map.json
Copy vocab file to t5-base-finetuned-bbc\spiece.model


Upload file pytorch_model.bin:   0%|          | 32.0k/850M [00:00<?, ?B/s]

Upload file runs/Oct29_17-29-30_ALIENMEDIA/1635499922.574325/events.out.tfevents.1635499922.ALIENMEDIA.13000.1…

Upload file runs/Oct29_17-29-30_ALIENMEDIA/events.out.tfevents.1635499922.ALIENMEDIA.13000.0: 100%|##########|…

Upload file runs/Oct29_17-37-24_ALIENMEDIA/1635500279.469721/events.out.tfevents.1635500279.ALIENMEDIA.13000.5…

...

Upload file runs/Oct29_18-17-58_ALIENMEDIA/events.out.tfevents.1635502690.ALIENMEDIA.5128.0: 100%|##########| …

Upload file runs/Oct29_17-40-51_ALIENMEDIA/events.out.tfevents.1635500472.ALIENMEDIA.13000.6: 100%|##########|…

Upload file training_args.bin: 100%|##########| 2.86k/2.86k [00:00<?, ?B/s]

To https://huggingface.co/furyhawk/t5-base-finetuned-bbc
   519baa1..a93787d  main -> main

Dropping the following result as it does not have all the necessary field:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/furyhawk/t5-base-finetuned-bbc
   a93787d..f587d43  main -> main



'https://huggingface.co/furyhawk/t5-base-finetuned-bbc/commit/a93787d3dd6bb3f018e3c338cfdbab5f3692e7c3'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```